In [ ]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from pyproj import Transformer

def load_json_file(filename):
    """Load a JSON file and return elements"""
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data.get('elements', [])

def extract_geometry(element):
    """Extract geometry from OSM element and return Shapely object in WGS84"""
    elem_type = element.get('type')
    
    if elem_type == 'node':
        # Point feature
        lat = element.get('lat')
        lon = element.get('lon')
        if lat and lon:
            return Point(lon, lat)
    
    elif elem_type == 'way':
        # LineString or Polygon
        geometry = element.get('geometry', [])
        if geometry and len(geometry) >= 3:
            coords = [(pt['lon'], pt['lat']) for pt in geometry]
            # Check if it's a closed polygon
            if len(coords) >= 4 and coords[0] == coords[-1]:
                return Polygon(coords)
            else:
                # Not closed - could be LineString or unclosed way
                # For ways, try closing it if it makes sense
                if len(coords) >= 3:
                    return Polygon(coords + [coords[0]])
                return LineString(coords)
    
    elif elem_type == 'relation':
        # Complex multipolygons
        geometry = element.get('geometry', [])
        if geometry and len(geometry) >= 3:
            coords = [(pt['lon'], pt['lat']) for pt in geometry]
            # Ensure closed
            if coords[0] != coords[-1]:
                coords.append(coords[0])
            if len(coords) >= 4:
                return Polygon(coords)
    
    return None

def get_feature_type(element):
    """Determine if element is park, wood, or forest"""
    tags = element.get('tags', {})
    
    if tags.get('leisure') == 'park':
        return 'park'
    elif tags.get('leisure') == 'garden':
        return 'garden'
    elif tags.get('natural') == 'wood':
        return 'wood'
    elif tags.get('landuse') == 'forest':
        return 'forest'
    else:
        return 'unknown'

def get_name(element):
    """Extract name from tags"""
    tags = element.get('tags', {})
    return tags.get('name', tags.get('name:en', ''))

def process_element(element, transformer):
    """Process a single OSM element with proper projection"""
    # Get geometry in WGS84 (EPSG:4326)
    geom_wgs84 = extract_geometry(element)
    if geom_wgs84 is None:
        return None
    
    # Transform to RD New (EPSG:28992) for accurate area calculation
    geom_rd = transform_geometry(geom_wgs84, transformer)
    
    # Calculate centroid in WGS84 (lat/lon)
    centroid_wgs84 = geom_wgs84.centroid
    
    # Calculate area in m² using RD New projection
    area_m2 = geom_rd.area if hasattr(geom_rd, 'area') else 0
    
    # Extract metadata
    osm_id = element.get('id')
    feature_type = get_feature_type(element)
    name = get_name(element)
    grid_cell = element.get('grid_cell', '')
    
    return {
        'osm_id': osm_id,
        'name': name,
        'type': feature_type,
        'centroid_lat': centroid_wgs84.y,
        'centroid_lon': centroid_wgs84.x,
        'area_m2': round(area_m2, 2),
        'grid_cell': grid_cell,
        'geometry_wkt': geom_wgs84.wkt  # Store WGS84 for reference
    }

def transform_geometry(geom, transformer):
    """Transform geometry from WGS84 to RD New"""
    if isinstance(geom, Point):
        x, y = transformer.transform(geom.y, geom.x)
        return Point(x, y)
    elif isinstance(geom, Polygon):
        exterior_coords = [transformer.transform(lat, lon) for lon, lat in geom.exterior.coords]
        return Polygon(exterior_coords)
    elif isinstance(geom, LineString):
        coords = [transformer.transform(lat, lon) for lon, lat in geom.coords]
        return LineString(coords)
    return geom

def main():
    print("Setting up coordinate transformer (WGS84 -> RD New)...")
    # Create transformer: WGS84 (EPSG:4326) -> RD New (EPSG:28992)
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:28992", always_xy=False)
    
    print("Loading JSON files...")
    
    # Load all files
    files = {
        '2020_parks': '../data/amsterdam_green_2020_parks_only.json',
        '2024_parks': '../data/amsterdam_green_2024_parks_only.json',
        '2020_wood_forest': '../data/amsterdam_green_2020_wood_forest.json',
        '2024_wood_forest': '../data/amsterdam_green_2024_wood_forest.json'
    }
    
    all_elements = []
    
    for file_key, filename in files.items():
        print(f"\nProcessing {filename}...")
        try:
            elements = load_json_file(filename)
            print(f"  Loaded {len(elements)} elements")
            
            # Add year and source info
            year = '2020' if '2020' in file_key else '2024'
            
            for elem in elements:
                elem['source_year'] = year
                elem['source_file'] = file_key
            
            all_elements.extend(elements)
            
        except FileNotFoundError:
            print(f"  Warning: File not found: {filename}")
        except Exception as e:
            print(f"  Warning: Error loading {filename}: {e}")
    
    print(f"\n{'='*60}")
    print(f"Total elements loaded: {len(all_elements)}")
    
    # Process all elements
    print("\nProcessing geometries and computing centroids...")
    processed = []
    failed = 0
    
    for i, elem in enumerate(all_elements):
        if (i + 1) % 500 == 0:
            print(f"  Processed {i + 1}/{len(all_elements)}...")
        
        try:
            result = process_element(elem, transformer)
            if result:
                result['source_year'] = elem.get('source_year')
                result['source_file'] = elem.get('source_file')
                processed.append(result)
            else:
                failed += 1
        except Exception as e:
            failed += 1
            if failed <= 5:  # Only print first few errors
                print(f"  Warning: Failed to process element {elem.get('id')}: {e}")
    
    print(f"  Successfully processed {len(processed)}/{len(all_elements)} elements")
    if failed > 0:
        print(f"  Failed: {failed} elements")
    
    # Create DataFrame
    df = pd.DataFrame(processed)
    
    # Display summary
    print(f"\n{'='*60}")
    print("SUMMARY:")
    print(f"{'='*60}")
    print(f"\nTotal green spaces: {len(df)}")
    print(f"\nBy type:")
    print(df['type'].value_counts())
    print(f"\nBy year:")
    print(df['source_year'].value_counts())
    print(f"\nBy grid cell:")
    print(df['grid_cell'].value_counts().sort_index())
    print(f"\nNamed features: {df['name'].notna().sum()}/{len(df)} ({100*df['name'].notna().sum()/len(df):.1f}%)")
    print(f"\nArea statistics:")
    print(f"  Total area: {df['area_m2'].sum():,.0f} m² ({df['area_m2'].sum()/1_000_000:,.2f} km²)")
    print(f"  Average: {df['area_m2'].mean():,.0f} m²")
    print(f"  Median: {df['area_m2'].median():,.0f} m²")
    print(f"  Largest: {df['area_m2'].max():,.0f} m²")
    
    # Show largest parks
    print(f"\nLargest green spaces:")
    print(df.nlargest(5, 'area_m2')[['name', 'type', 'area_m2', 'source_year']])
    
    # Save to CSV
    output_file = '../data/green_areas.csv'
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"\n✓ Saved to {output_file}")
    
    # Show sample
    print(f"\nSample rows:")
    cols_to_show = ['osm_id', 'name', 'type', 'centroid_lat', 'centroid_lon', 'area_m2', 'grid_cell', 'source_year']
    print(df[cols_to_show].head(10).to_string())
    
    return df

if __name__ == "__main__":
    df = main()

Setting up coordinate transformer (WGS84 -> RD New)...
Loading JSON files...

Processing amsterdam_green_2020_parks_only.json...
  Loaded 322 elements

Processing amsterdam_green_2024_parks_only.json...
  Loaded 377 elements

Processing amsterdam_green_2020_wood_forest.json...
  Loaded 8600 elements

Processing amsterdam_green_2024_wood_forest.json...
  Loaded 10693 elements

Total elements loaded: 19992

Processing geometries and computing centroids...
  Processed 500/19992...
  Processed 1000/19992...
  Processed 1500/19992...
  Processed 2000/19992...
  Processed 2500/19992...
  Processed 3000/19992...
  Processed 3500/19992...
  Processed 4000/19992...
  Processed 4500/19992...
  Processed 5000/19992...
  Processed 5500/19992...
  Processed 6000/19992...
  Processed 6500/19992...
  Processed 7000/19992...
  Processed 7500/19992...
  Processed 8000/19992...
  Processed 8500/19992...
  Processed 9000/19992...
  Processed 9500/19992...
  Processed 10000/19992...
  Processed 10500/1999

In [ ]:
# Load the CSV
df = pd.read_csv('../data/green_areas.csv')

print("Original data:")
print(f"Total rows: {len(df)}")
print(f"\nBreakdown by year:")
print(df['source_year'].value_counts())

# Check for duplicates based on osm_id and year
print(f"\nDuplicates based on (osm_id, source_year):")
duplicates = df[df.duplicated(subset=['osm_id', 'source_year'], keep=False)]
print(f"Found {len(duplicates)} duplicate rows")

if len(duplicates) > 0:
    print("\nSample duplicates:")
    print(duplicates[['osm_id', 'name', 'type', 'source_year', 'source_file']].head(20))

# Remove duplicates - keep first occurrence
df_clean = df.drop_duplicates(subset=['osm_id', 'source_year'], keep='first')

print(f"\n{'='*60}")
print("After deduplication:")
print(f"Total rows: {len(df_clean)}")
print(f"Removed: {len(df) - len(df_clean)} duplicate rows")
print(f"\nBreakdown by year:")
print(df_clean['source_year'].value_counts())
print(f"\nBreakdown by type:")
print(df_clean['type'].value_counts())


# Save cleaned data
output_file = '../data/green_areas_clean.csv'
df_clean.to_csv(output_file, index=False, encoding='utf-8')
print(f"\n✓ Saved deduplicated data to {output_file}")

# Summary statistics
print(f"\n{'='*60}")
print("Area statistics (cleaned data):")
print(f"  Total area: {df_clean['area_m2'].sum():,.0f} m² ({df_clean['area_m2'].sum()/1_000_000:,.2f} km²)")
print(f"  Average: {df_clean['area_m2'].mean():,.0f} m²")
print(f"  Median: {df_clean['area_m2'].median():,.0f} m²")

Original data:
Total rows: 19838

Breakdown by year:
source_year
2024    10997
2020     8841
Name: count, dtype: int64

Duplicates based on (osm_id, source_year):
Found 628 duplicate rows

Sample duplicates:
        osm_id                     name  type  source_year source_file
1     26517033          Amsterdamse Bos  park         2020  2020_parks
2     46785402       Sportpark Overburg  park         2020  2020_parks
11    98133437  Dr. Jac. P. Thijssepark  park         2020  2020_parks
19   311453250             Seifertplein  park         2020  2020_parks
22   403847003         Stadspark Osdorp  park         2020  2020_parks
25   485022427        Kersenbloesempark  park         2020  2020_parks
29   536787142         Piet Wiedijkpark  park         2020  2020_parks
33     6316270             Sarphatipark  park         2020  2020_parks
43    26517033          Amsterdamse Bos  park         2020  2020_parks
65    44213014               Vondelpark  park         2020  2020_parks
68    46785

In [ ]:
from shapely import wkt

# Load the data
print("Loading data...")
green_df = pd.read_csv('../data/green_areas_clean.csv')
buurten_df = pd.read_csv('../data/buurten.csv')

print(f"Green spaces: {len(green_df)} entries")

# DEBUG: Check what source_year values actually are
print("\nDEBUG - source_year info:")
print(f"  Unique values: {green_df['source_year'].unique()}")
print(f"  Data type: {green_df['source_year'].dtype}")
print(f"  Sample rows:")
print(green_df[['osm_id', 'name', 'type', 'source_year']].head())

# Convert source_year to string for consistency
green_df['source_year'] = green_df['source_year'].astype(str)

print(f"\nAfter conversion:")
print(f"  2020: {len(green_df[green_df['source_year'] == '2020'])} entries")
print(f"  2024: {len(green_df[green_df['source_year'] == '2024'])} entries")
print(f"Buurten: {len(buurten_df)} entries")

# Parse buurt geometry
def parse_buurt_geometry(geom_string):
    if pd.isna(geom_string):
        return None
    clean_wkt = geom_string.split(';')[-1]
    return wkt.loads(clean_wkt)

buurten_df['polygon_geom'] = buurten_df['geometrie'].apply(parse_buurt_geometry)
buurten_gdf = gpd.GeoDataFrame(buurten_df, geometry='polygon_geom', crs='EPSG:28992')

# Create centroid points for buurten
buurten_df['centroid_geom'] = buurten_df.apply(
    lambda row: Point(row['longitude'], row['latitude']), axis=1
)
buurten_centroids_gdf = gpd.GeoDataFrame(buurten_df, geometry='centroid_geom', crs='EPSG:4326')
buurten_centroids_rd = buurten_centroids_gdf.to_crs('EPSG:28992')

def calculate_metrics_for_year(year):
    """Calculate green metrics for a specific year"""
    print(f"\n{'='*60}")
    print(f"CALCULATING METRICS FOR {year}")
    print(f"{'='*60}")
    
    # Filter green spaces for this year
    green_year = green_df[green_df['source_year'] == year].copy()
    print(f"Using {len(green_year)} green spaces from {year}")
    
    if len(green_year) == 0:
        print(f"ERROR: No green spaces found for year {year}!")
        return None
    
    # Convert to GeoDataFrame
    green_year['geometry'] = green_year['geometry_wkt'].apply(wkt.loads)
    green_gdf = gpd.GeoDataFrame(green_year, geometry='geometry', crs='EPSG:4326')
    green_gdf_rd = green_gdf.to_crs('EPSG:28992')
    
    # Create green centroids
    green_centroids_gdf = gpd.GeoDataFrame(
        green_year[['osm_id', 'name', 'type', 'area_m2']],
        geometry=gpd.points_from_xy(green_year['centroid_lon'], green_year['centroid_lat']),
        crs='EPSG:4326'
    )
    green_centroids_rd = green_centroids_gdf.to_crs('EPSG:28992')
    
    results = []
    
    for idx in range(len(buurten_gdf)):
        if (idx + 1) % 50 == 0:
            print(f"  Processing {idx + 1}/{len(buurten_gdf)}...")
        
        buurt_row = buurten_gdf.iloc[idx]
        buurt_polygon = buurten_gdf.geometry.iloc[idx]
        buurt_centroid = buurten_centroids_rd.geometry.iloc[idx]
        
        buurt_area = buurt_polygon.area if hasattr(buurt_polygon, 'area') else 0
        
        # METRIC 1: Distance to nearest green space
        distances = green_centroids_rd.geometry.distance(buurt_centroid)
        min_distance = distances.min()
        nearest_idx = distances.idxmin()
        nearest_green = green_centroids_rd.loc[nearest_idx]
        
        # METRIC 2: Total green area within buurt
        intersecting_greens = green_gdf_rd[green_gdf_rd.intersects(buurt_polygon)]
        
        total_green_area = 0
        if len(intersecting_greens) > 0:
            for _, green in intersecting_greens.iterrows():
                try:
                    intersection = green.geometry.intersection(buurt_polygon)
                    if not intersection.is_empty:
                        total_green_area += intersection.area
                except:
                    continue
        
        total_green_area = min(total_green_area, buurt_area)
        green_coverage_percent = (total_green_area / buurt_area * 100) if buurt_area > 0 else 0
        
        result = {
            'BUURTCODE_2022': buurt_row['BUURTCODE_2022'],
            f'distance_to_nearest_green_m_{year}': round(min_distance, 2),
            f'nearest_green_name_{year}': nearest_green['name'] if pd.notna(nearest_green['name']) else 'Unnamed',
            f'nearest_green_type_{year}': nearest_green['type'],
            f'nearest_green_area_m2_{year}': nearest_green['area_m2'],
            f'total_green_area_m2_{year}': round(total_green_area, 2),
            f'green_coverage_percent_{year}': round(green_coverage_percent, 2),
            f'num_green_spaces_{year}': len(intersecting_greens)
        }
        
        results.append(result)
    
    return pd.DataFrame(results)

# Calculate for both years
results_2020 = calculate_metrics_for_year('2020')
results_2024 = calculate_metrics_for_year('2024')

if results_2020 is None or results_2024 is None:
    print("\nERROR: Could not calculate metrics. Check source_year values in green_areas_clean.csv")
else:
    # Merge into single dataframe
    merged_results = results_2020.merge(results_2024, on='BUURTCODE_2022')
    
    # Add buurt metadata
    buurt_metadata = buurten_df[['BUURTCODE_2022', 'BUURTNAAM_2022', 'WIJKNAAM_2022', 'STADSDEELNAAM']].copy()
    final_results = buurt_metadata.merge(merged_results, on='BUURTCODE_2022')
    
    # Add buurt area
    final_results['buurt_area_m2'] = buurten_gdf.geometry.area.round(2).values
    
    # Calculate changes
    final_results['distance_change_m'] = final_results['distance_to_nearest_green_m_2024'] - final_results['distance_to_nearest_green_m_2020']
    final_results['coverage_change_percent'] = final_results['green_coverage_percent_2024'] - final_results['green_coverage_percent_2020']
    
    # Reorder columns
    column_order = [
        'BUURTCODE_2022', 'BUURTNAAM_2022', 'WIJKNAAM_2022', 'STADSDEELNAAM', 'buurt_area_m2',
        'distance_to_nearest_green_m_2020', 'distance_to_nearest_green_m_2024', 'distance_change_m',
        'nearest_green_name_2020', 'nearest_green_type_2020', 'nearest_green_area_m2_2020',
        'nearest_green_name_2024', 'nearest_green_type_2024', 'nearest_green_area_m2_2024',
        'total_green_area_m2_2020', 'total_green_area_m2_2024',
        'green_coverage_percent_2020', 'green_coverage_percent_2024', 'coverage_change_percent',
        'num_green_spaces_2020', 'num_green_spaces_2024'
    ]
    final_results = final_results[column_order]
    
    # Save to CSV
    output_file = '../data/buurt_green_metrics.csv'
    final_results.to_csv(output_file, index=False, encoding='utf-8')
    print(f"\n✓ Saved results to {output_file}")
    
    # Print summary statistics
    print(f"\n{'='*60}")
    print("SUMMARY STATISTICS")
    print(f"{'='*60}")
    
    for year in ['2020', '2024']:
        print(f"\n{year}:")
        print(f"  Avg distance to green: {final_results[f'distance_to_nearest_green_m_{year}'].mean():.2f} m")
        print(f"  Median distance: {final_results[f'distance_to_nearest_green_m_{year}'].median():.2f} m")
        print(f"  Avg green coverage: {final_results[f'green_coverage_percent_{year}'].mean():.2f}%")
        print(f"  Median green coverage: {final_results[f'green_coverage_percent_{year}'].median():.2f}%")
        print(f"  Buurten with 0% coverage: {(final_results[f'green_coverage_percent_{year}'] == 0).sum()}")
        print(f"  Buurten with >50% coverage: {(final_results[f'green_coverage_percent_{year}'] > 50).sum()}")
    
    print(f"\n{'='*60}")
    print("CHANGES FROM 2020 TO 2024")
    print(f"{'='*60}")
    print(f"\nAverage distance change: {final_results['distance_change_m'].mean():.2f} m")
    print(f"Average coverage change: {final_results['coverage_change_percent'].mean():.2f}%")
    
    print(f"\nBuurten with largest coverage INCREASE:")
    print(final_results.nlargest(5, 'coverage_change_percent')[
        ['BUURTNAAM_2022', 'STADSDEELNAAM', 'green_coverage_percent_2020', 'green_coverage_percent_2024', 'coverage_change_percent']
    ].to_string(index=False))
    
    print(f"\nBuurten with largest coverage DECREASE:")
    print(final_results.nsmallest(5, 'coverage_change_percent')[
        ['BUURTNAAM_2022', 'STADSDEELNAAM', 'green_coverage_percent_2020', 'green_coverage_percent_2024', 'coverage_change_percent']
    ].to_string(index=False))
    
    print(f"\nBuurten FURTHEST from green in 2024:")
    print(final_results.nlargest(10, 'distance_to_nearest_green_m_2024')[
        ['BUURTNAAM_2022', 'STADSDEELNAAM', 'distance_to_nearest_green_m_2024', 'green_coverage_percent_2024']
    ].to_string(index=False))
    
    print(f"\nBuurten with LOWEST green coverage in 2024:")
    print(final_results.nsmallest(10, 'green_coverage_percent_2024')[
        ['BUURTNAAM_2022', 'STADSDEELNAAM', 'green_coverage_percent_2024', 'distance_to_nearest_green_m_2024']
    ].to_string(index=False))

Loading data...
Green spaces: 19523 entries

DEBUG - source_year info:
  Unique values: [2020 2024]
  Data type: int64
  Sample rows:
     osm_id                name  type  source_year
0  26493165       Kasterleepark  park         2020
1  26517033     Amsterdamse Bos  park         2020
2  46785402  Sportpark Overburg  park         2020
3  58169403                 NaN  park         2020
4  62923595          Wandelpark  park         2020

After conversion:
  2020: 8687 entries
  2024: 10836 entries
Buurten: 439 entries

CALCULATING METRICS FOR 2020
Using 8687 green spaces from 2020
  Processing 50/439...
  Processing 100/439...
  Processing 150/439...
  Processing 200/439...
  Processing 250/439...
  Processing 300/439...
  Processing 350/439...
  Processing 400/439...

CALCULATING METRICS FOR 2024
Using 10836 green spaces from 2024
  Processing 50/439...
  Processing 100/439...
  Processing 150/439...
  Processing 200/439...
  Processing 250/439...
  Processing 300/439...
  Processing 3

### preparing socio economic data

In [ ]:
import re

print("="*60)
print("STEP 1: Loading CSV files")
print("="*60)

# Load the three CSV files
df_2024 = pd.read_csv('../data/Kerncijfers_wijken_en_buurten_2024_without_income_data.csv', sep=';')
df_2023 = pd.read_csv('../data/Kerncijfers_wijken_en_buurten_2023_income.csv', sep=';')
df_2020 = pd.read_csv('../data/Kerncijfers_wijken_en_buurten_2020.csv', sep=';')

print(f"2024 file: {df_2024.shape[0]} rows, {df_2024.shape[1]} columns")
print(f"2023 file: {df_2023.shape[0]} rows, {df_2023.shape[1]} columns")
print(f"2020 file: {df_2020.shape[0]} rows, {df_2020.shape[1]} columns")

print("\n" + "="*60)
print("STEP 2: Cleaning whitespace and quotes from columns 2 and 3")
print("="*60)

def clean_column(df, col_name):
    """Remove trailing whitespace and quotes from a column"""
    if col_name in df.columns:
        df[col_name] = df[col_name].astype(str).str.strip().str.strip('"').str.strip("'")
    return df

# Clean each dataframe using its own column names
for df in [df_2024, df_2023, df_2020]:
    # Clean columns 1 and 2 (0-indexed)
    if len(df.columns) > 1:
        clean_column(df, df.columns[1])
    if len(df.columns) > 2:
        clean_column(df, df.columns[2])

print(f"✓ Cleaned columns 2 and 3 in all files")
print(f"\nSample column names:")
print(f"  df_2024 col 2: '{df_2024.columns[1]}'")
print(f"  df_2023 col 2: '{df_2023.columns[1]}'")
print(f"  df_2020 col 2: '{df_2020.columns[1]}'")


print("\n" + "="*60)
print("STEP 3: Merging 2023 income data into 2024 file")
print("="*60)

# Identify columns to merge from 2023 (income and education columns)
columns_to_skip = [df_2023.columns[0], df_2023.columns[1], df_2023.columns[2]]  # First 3 columns
income_education_cols = [col for col in df_2023.columns if col not in columns_to_skip]

print(f"Columns to merge from 2023 income file: {len(income_education_cols)}")
print(f"Sample columns: {income_education_cols[:3]}")

# Merge data based on encoding column (3rd column)
encoding_col = df_2024.columns[2]

# Create a mapping from 2023 data
for col in income_education_cols:
    # Create dictionary mapping encoding to values
    value_map = dict(zip(df_2023[encoding_col], df_2023[col]))
    
    # If column exists in 2024, overwrite it; otherwise create it
    if col in df_2024.columns:
        df_2024[col] = df_2024[encoding_col].map(value_map)
    else:
        df_2024.insert(len(df_2024.columns), col, df_2024[encoding_col].map(value_map))

print(f"✓ Merged {len(income_education_cols)} columns from 2023 into 2024")
print(f"2024 file now has {df_2024.shape[1]} columns")

print("\n" + "="*60)
print("STEP 4: Translating Dutch headers to English")
print("="*60)

# Translation dictionary
translation_dict = {
    "Wijken en buurten": "District_Type",
    "Regioaanduiding/Gemeentenaam (naam)": "Municipality_Name",
    "Regioaanduiding/Codering (code)": "Area_Code",
    "Bevolking/Aantal inwoners (aantal)": "Population_Total",
    "Bevolking/Leeftijdsgroepen/0 tot 15 jaar (aantal)": "Pop_Age_0_15",
    "Bevolking/Leeftijdsgroepen/15 tot 25 jaar (aantal)": "Pop_Age_15_25",
    "Bevolking/Leeftijdsgroepen/25 tot 45 jaar (aantal)": "Pop_Age_25_45",
    "Bevolking/Leeftijdsgroepen/45 tot 65 jaar (aantal)": "Pop_Age_45_65",
    "Bevolking/Leeftijdsgroepen/65 jaar of ouder (aantal)": "Pop_Age_65_Plus",
    "Bevolking/Particuliere huishoudens/Gemiddelde huishoudensgrootte (aantal)": "Avg_Household_Size",
    "Wonen/Gemiddelde WOZ-waarde van woningen (x 1 000 euro)": "Avg_Property_Value_1000EUR",
    "Wonen/Woningen naar bewoning/Percentage onbewoond (%)": "Pct_Unoccupied_Homes",
    "Wonen/Woningen naar eigendom/Koopwoningen (%)": "Pct_Owner_Occupied",
    "Wonen/Woningen naar eigendom/Huurwoningen/Huurwoningen totaal (%)": "Pct_Rental",
    "Wonen en vastgoed/Gemiddelde WOZ-waarde van woningen (x 1 000 euro)": "Avg_Property_Value_1000EUR",
    "Wonen en vastgoed/Onbewoonde woningen (%)": "Pct_Unoccupied_Homes",
    "Wonen en vastgoed/Woningen naar eigendom/Koopwoningen (%)": "Pct_Owner_Occupied",
    "Wonen en vastgoed/Woningen naar eigendom/Huurwoningen/Huurwoningen totaal (%)": "Pct_Rental",
    "Opleidingsniveau/Opleidingsniveau hoog  (aantal)": "Higher_Education_Count",
    "Onderwijs/Hoogst behaald onderwijsniveau/Hbo, wo (aantal)": "Higher_Education_Count",
    "Inkomen/Inkomen van personen/Gemiddeld inkomen per inkomensontvanger  (x 1 000 euro)": "Avg_Income_Per_Earner_1000EUR",
    "Inkomen/Inkomen van personen/Gemiddeld inkomen per inwoner  (x 1 000 euro)": "Avg_Income_Per_Resident_1000EUR",
    "Inkomen/Inkomen van huishoudens/Gem. gestandaardiseerd inkomen van huish (x 1 000 euro)": "Avg_Std_Household_Income_1000EUR",
    "Inkomen/Inkomen van huishoudens/40% huishoudens met laagste inkomen (%)": "Pct_Lowest_40Pct_Income",
    "Inkomen/Inkomen van huishoudens/20% huishoudens met hoogste inkomen (%)": "Pct_Highest_20Pct_Income",
    "Inkomen/Inkomen van huishoudens/Huishoudens met een laag inkomen (%)": "Pct_Low_Income_Households",
    "Inkomen/Inkomen van huishoudens/Huishoudens tot 120% van sociaal minimum (%)": "Pct_Below_120Pct_Social_Min",
    "Stedelijkheid/Mate van stedelijkheid (code)": "Urbanization_Level"
}

def translate_columns(df, trans_dict):
    """Translate column names from Dutch to English"""
    new_columns = []
    for col in df.columns:
        if col in trans_dict:
            new_columns.append(trans_dict[col])
        else:
            # Keep original if no translation found
            new_columns.append(col)
    df.columns = new_columns
    return df

df_2024_translated = translate_columns(df_2024.copy(), translation_dict)
df_2020_translated = translate_columns(df_2020.copy(), translation_dict)

print("✓ Translated column names")
print(f"\nSample translations:")
print(f"  Original: {df_2024.columns[3]}")
print(f"  Translated: {df_2024_translated.columns[3]}")

print("\n" + "="*60)
print("STEP 5: Saving processed files")
print("="*60)

# Save 2024 file with new name (removing "without_income_data")
output_2024 = '../data/Kerncijfers_wijken_en_buurten_2024.csv'
df_2024_translated.to_csv(output_2024, index=False, sep=';')
print(f"✓ Saved: {output_2024}")

# Save 2020 file with translated headers
output_2020 = '../data/Kerncijfers_wijken_en_buurten_2020.csv'
df_2020_translated.to_csv(output_2020, index=False, sep=';')
print(f"✓ Saved: {output_2020}")

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"\nProcessed files:")
print(f"  1. {output_2024}")
print(f"     - {df_2024_translated.shape[0]} rows x {df_2024_translated.shape[1]} columns")
print(f"     - Includes 2023 income/education data")
print(f"\n  2. {output_2020}")
print(f"     - {df_2020_translated.shape[0]} rows x {df_2020_translated.shape[1]} columns")
print(f"     - Translated column names")

print(f"\nColumn names (first 10):")
for i, col in enumerate(df_2024_translated.columns[:10]):
    print(f"  {i+1}. {col}")

print("\n✓ All tasks completed successfully!")

STEP 1: Loading CSV files
2024 file: 628 rows, 23 columns
2023 file: 628 rows, 11 columns
2020 file: 579 rows, 23 columns

STEP 2: Cleaning whitespace and quotes from columns 2 and 3
✓ Cleaned columns 2 and 3 in all files

Sample column names:
  df_2024 col 2: 'Municipality_Name'
  df_2023 col 2: 'Municipality_Name'
  df_2020 col 2: 'Municipality_Name'

STEP 3: Merging 2023 income data into 2024 file
Columns to merge from 2023 income file: 8
Sample columns: ['Higher_Education_Count', 'Avg_Income_Per_Income_Recipient_1000EUR', 'Avg_Income_Per_Inhabitant_1000EUR']
✓ Merged 8 columns from 2023 into 2024
2024 file now has 23 columns

STEP 4: Translating Dutch headers to English
✓ Translated column names

Sample translations:
  Original: Population_Total
  Translated: Population_Total

STEP 5: Saving processed files
✓ Saved: Kerncijfers_wijken_en_buurten_2024.csv
✓ Saved: Kerncijfers_wijken_en_buurten_2020.csv

SUMMARY

Processed files:
  1. Kerncijfers_wijken_en_buurten_2024.csv
     - 628

In [ ]:
print("\n" + "="*60)
print("STEP 6: Filling missing buurt income data with wijk values")
print("="*60)

def extract_wijk_code(area_code):
    """
    Extract wijk code from buurt code.
    BU03630300 -> WK036303
    WK036303 -> WK036303 (already a wijk)
    """
    if pd.isna(area_code):
        return None
    area_code = str(area_code).strip()
    if area_code.startswith('WK'):
        return area_code
    elif area_code.startswith('BU') and len(area_code) >= 8:
        # BU03630300 -> WK036303 (first 8 characters, replace BU with WK)
        return 'WK' + area_code[2:8]
    return None

def fill_buurt_with_wijk_income(df, income_columns):
    """
    Fill missing buurt income values with their wijk values if available.
    """
    # Create wijk code column
    df['wijk_code'] = df['Area_Code'].apply(extract_wijk_code)
    
    # Create dictionary mapping wijk codes to their income values
    wijk_data = df[df['Area_Code'].str.startswith('WK', na=False)].copy()
    
    filled_count = {col: 0 for col in income_columns}
    
    for col in income_columns:
        # Create mapping: wijk_code -> income_value
        wijk_income_map = dict(zip(wijk_data['Area_Code'], wijk_data[col]))
        
        # For each row (buurt), if income is missing, try to fill with wijk value
        for idx, row in df.iterrows():
            if pd.isna(row[col]) or row[col] == '' or row[col] == '.':
                wijk_code = row['wijk_code']
                if wijk_code and wijk_code in wijk_income_map:
                    wijk_value = wijk_income_map[wijk_code]
                    if pd.notna(wijk_value) and wijk_value != '' and wijk_value != '.':
                        df.at[idx, col] = wijk_value
                        filled_count[col] += 1
    
    # Drop temporary wijk_code column
    df = df.drop(columns=['wijk_code'])
    
    return df, filled_count

# Income columns to fill
income_columns = [
    'Avg_Income_Per_Income_Recipient_1000EUR',
    'Avg_Income_Per_Inhabitant_1000EUR', 
    'Avg_Std_Household_Income_1000EUR',
    'Pct_Lowest_40Pct_Income',
    'Pct_Highest_20Pct_Income',
    'Pct_Low_Income_Households',
    'Pct_Below_120Pct_Social_Min',
]

print("\nProcessing 2024 file...")
print("-" * 60)

# Check missing values before
missing_before_2024 = {}
for col in income_columns:
    missing_before_2024[col] = df_2024_translated[col].isna().sum()

print(f"\nMissing values BEFORE filling (2024):")
for col, count in missing_before_2024.items():
    print(f"  {col}: {count}")

# Fill missing values
df_2024_filled, filled_2024 = fill_buurt_with_wijk_income(df_2024_translated.copy(), income_columns)

print(f"\nValues filled from wijk data (2024):")
for col, count in filled_2024.items():
    print(f"  {col}: {count} buurten filled")

# Check missing values after
missing_after_2024 = {}
for col in income_columns:
    missing_after_2024[col] = df_2024_filled[col].isna().sum()

print(f"\nMissing values AFTER filling (2024):")
for col, count in missing_after_2024.items():
    print(f"  {col}: {count}")

print("\n" + "-" * 60)
print("Processing 2020 file...")
print("-" * 60)

# Check missing values before
missing_before_2020 = {}
for col in income_columns:
    missing_before_2020[col] = df_2020_translated[col].isna().sum()

print(f"\nMissing values BEFORE filling (2020):")
for col, count in missing_before_2020.items():
    print(f"  {col}: {count}")

# Fill missing values
df_2020_filled, filled_2020 = fill_buurt_with_wijk_income(df_2020_translated.copy(), income_columns)

print(f"\nValues filled from wijk data (2020):")
for col, count in filled_2020.items():
    print(f"  {col}: {count} buurten filled")

# Check missing values after
missing_after_2020 = {}
for col in income_columns:
    missing_after_2020[col] = df_2020_filled[col].isna().sum()

print(f"\nMissing values AFTER filling (2020):")
for col, count in missing_after_2020.items():
    print(f"  {col}: {count}")

print("\n" + "="*60)
print("STEP 7: Saving updated files")
print("="*60)

# Save updated files
df_2024_filled.to_csv('../data/Kerncijfers_wijken_en_buurten_2024.csv', index=False, sep=';')
print(f"✓ Updated and saved: Kerncijfers_wijken_en_buurten_2024.csv")

df_2020_filled.to_csv('../data/Kerncijfers_wijken_en_buurten_2020.csv', index=False, sep=';')
print(f"✓ Updated and saved: Kerncijfers_wijken_en_buurten_2020.csv")

print("\n" + "="*60)
print("SUMMARY OF INCOME DATA FILLING")
print("="*60)

print("\n2024 file:")
for col in income_columns:
    before = missing_before_2024[col]
    filled = filled_2024[col]
    after = missing_after_2024[col]
    print(f"\n  {col}:")
    print(f"    Missing before: {before}")
    print(f"    Filled with wijk: {filled}")
    print(f"    Still missing: {after}")
    print(f"    Improvement: {before - after} fewer missing values")

print("\n2020 file:")
for col in income_columns:
    before = missing_before_2020[col]
    filled = filled_2020[col]
    after = missing_after_2020[col]
    print(f"\n  {col}:")
    print(f"    Missing before: {before}")
    print(f"    Filled with wijk: {filled}")
    print(f"    Still missing: {after}")
    print(f"    Improvement: {before - after} fewer missing values")

print("\n✓ Income data filling completed!")
print("\nNote: Remaining missing values are buurten where even the wijk has no data.")


STEP 6: Filling missing buurt income data with wijk values

Processing 2024 file...
------------------------------------------------------------

Missing values BEFORE filling (2024):
  Avg_Income_Per_Income_Recipient_1000EUR: 433
  Avg_Income_Per_Inhabitant_1000EUR: 384
  Avg_Std_Household_Income_1000EUR: 526
  Pct_Lowest_40Pct_Income: 98
  Pct_Highest_20Pct_Income: 98
  Pct_Low_Income_Households: 105
  Pct_Below_120Pct_Social_Min: 105

Values filled from wijk data (2024):
  Avg_Income_Per_Income_Recipient_1000EUR: 357 buurten filled
  Avg_Income_Per_Inhabitant_1000EUR: 308 buurten filled
  Avg_Std_Household_Income_1000EUR: 410 buurten filled
  Pct_Lowest_40Pct_Income: 91 buurten filled
  Pct_Highest_20Pct_Income: 91 buurten filled
  Pct_Low_Income_Households: 89 buurten filled
  Pct_Below_120Pct_Social_Min: 89 buurten filled

Missing values AFTER filling (2024):
  Avg_Income_Per_Income_Recipient_1000EUR: 76
  Avg_Income_Per_Inhabitant_1000EUR: 76
  Avg_Std_Household_Income_1000EUR: 

### cleaning datasets (only keep valid buurten entries)

In [ ]:
# Load the reference file with valid buurten codes
print("Loading reference file: buurten.csv")
buurten_df = pd.read_csv('../data/buurten.csv')

# Get CBS codes (for most files)
valid_buurt_codes_2015_cbs = set(buurten_df['BUURTCODE_CBS_2015'].unique())
valid_buurt_codes_2022_cbs = set(buurten_df['BUURTCODE_CBS_2022'].unique())
all_valid_codes_cbs = valid_buurt_codes_2015_cbs.union(valid_buurt_codes_2022_cbs)

# Get regular buurt codes (for buurt_green_metrics.csv)
valid_buurt_codes_2015_regular = set(buurten_df['BUURTCODE_2015'].unique())
valid_buurt_codes_2022_regular = set(buurten_df['BUURTCODE_2022'].unique())
all_valid_codes_regular = valid_buurt_codes_2015_regular.union(valid_buurt_codes_2022_regular)

print(f"Found {len(valid_buurt_codes_2015_cbs)} valid 2015 CBS buurt codes")
print(f"Found {len(valid_buurt_codes_2022_cbs)} valid 2022 CBS buurt codes")
print(f"Total unique valid CBS codes: {len(all_valid_codes_cbs)}")
print(f"\nFound {len(valid_buurt_codes_2015_regular)} valid 2015 regular buurt codes")
print(f"Found {len(valid_buurt_codes_2022_regular)} valid 2022 regular buurt codes")
print(f"Total unique valid regular codes: {len(all_valid_codes_regular)}")
print(f"\nSample CBS codes: {list(all_valid_codes_cbs)[:5]}")
print(f"Sample regular codes: {list(all_valid_codes_regular)[:5]}")

# Define the files to filter
files_to_filter = {
    '../data/buurt_green_metrics.csv': {
        'code_column': 'BUURTCODE_2022',
        'separator': ',',
        'year': 2022,
        'use_regular_codes': True  # Use BUURTCODE columns instead of BUURTCODE_CBS
    },
    '../data/Kerncijfers_wijken_en_buurten_2020.csv': {
        'code_column': 'Area_Code',
        'separator': ';',
        'year': 2020,
        'use_regular_codes': False
    },
    '../data/Kerncijfers_wijken_en_buurten_2024.csv': {
        'code_column': 'Area_Code',
        'separator': ';',
        'year': 2024,
        'use_regular_codes': False
    },
    '../data/proximity_to_facilities_2020.csv': {
        'code_column': 'Encoding',
        'separator': ';',
        'year': 2020,
        'use_regular_codes': False
    },
    '../data/proximity_to_facilities_2024.csv': {
        'code_column': 'Encoding_3',
        'separator': ';',
        'year': 2024,
        'use_regular_codes': False
    }
}

# Process each file
summary = []

for filename, config in files_to_filter.items():
    print(f"\n{'='*60}")
    print(f"Processing: {filename}")
    print(f"{'='*60}")
    
    try:
        # Read file with appropriate separator
        df = pd.read_csv(filename, sep=config['separator'])
        
        original_count = len(df)
        code_column = config['code_column']
        file_year = config['year']
        
        # Select appropriate validation codes
        if config.get('use_regular_codes', False):
            all_valid_codes = all_valid_codes_regular
            codes_2015 = valid_buurt_codes_2015_regular
            codes_2022 = valid_buurt_codes_2022_regular
            code_type_name = "regular buurt codes"
        else:
            all_valid_codes = all_valid_codes_cbs
            codes_2015 = valid_buurt_codes_2015_cbs
            codes_2022 = valid_buurt_codes_2022_cbs
            code_type_name = "CBS buurt codes"
        
        print(f"Original rows: {original_count}")
        print(f"File year: {file_year}")
        print(f"Using: {code_type_name}")
        
        # Check if code column exists
        if code_column not in df.columns:
            print(f"ERROR: Column '{code_column}' not found!")
            print(f"Available columns: {df.columns.tolist()}")
            continue
        
        # Strip whitespace from codes for comparison
        df[code_column] = df[code_column].astype(str).str.strip()
        
        # Filter to only valid buurt codes
        filtered_df = df[df[code_column].isin(all_valid_codes)]
        
        filtered_count = len(filtered_df)
        removed_count = original_count - filtered_count
        
        print(f"Filtered rows: {filtered_count}")
        print(f"Removed rows: {removed_count}")
        
        if removed_count > 0:
            # Show some examples of removed codes
            removed_codes = set(df[code_column]) - all_valid_codes
            print(f"\nSample removed codes (first 10):")
            for code in list(removed_codes)[:10]:
                # Check if it's a wijk/gemeente code by looking at pattern
                code_type = "Unknown"
                if code.startswith('GM'):
                    code_type = "Municipality (GM)"
                elif code.startswith('WK'):
                    code_type = "Wijk (WK)"
                elif code.startswith('BU'):
                    code_type = "Buurt (BU) - but not in reference"
                print(f"  - {code} [{code_type}]")
        
        # Save filtered file
        base_filename = filename.split('/')[-1] # Get filename without path
        output_filename = f"../data/filtered_{base_filename}"
        filtered_df.to_csv(output_filename, sep=config['separator'], index=False)
        
        print(f"✓ Saved filtered data to: {output_filename}")
        
        # Additional statistics
        if filtered_count > 0:
            # Check which code system is being used
            codes_in_2015 = filtered_df[code_column].isin(codes_2015).sum()
            codes_in_2022 = filtered_df[code_column].isin(codes_2022).sum()
            print(f"\nCode distribution:")
            print(f"  Matches 2015 codes: {codes_in_2015}")
            print(f"  Matches 2022 codes: {codes_in_2022}")
        
        summary.append({
            'File': filename,
            'Year': file_year,
            'Code_Type': code_type_name,
            'Original': original_count,
            'Filtered': filtered_count,
            'Removed': removed_count,
            'Percentage_Kept': f"{(filtered_count/original_count*100):.1f}%"
        })
        
    except Exception as e:
        print(f"ERROR processing {filename}: {str(e)}")
        continue

# Print summary
print(f"\n{'='*60}")
print("SUMMARY")
print(f"{'='*60}\n")
summary_df = pd.DataFrame(summary)
print(summary_df.to_string(index=False))

print(f"\n{'='*60}")
print("VALIDATION")
print(f"{'='*60}")
print("\nChecking code consistency across files...")

# Load filtered files and check overlap
try:
    green_2020 = pd.read_csv('../data/filtered_buurt_green_metrics.csv')
    kern_2020 = pd.read_csv('../data/filtered_Kerncijfers_wijken_en_buurten_2020.csv', sep=';')
    kern_2024 = pd.read_csv('../data/filtered_Kerncijfers_wijken_en_buurten_2024.csv', sep=';')
    
    green_codes = set(green_2020['BUURTCODE_2022'].unique())
    kern_2020_codes = set(kern_2020['Area_Code'].unique())
    kern_2024_codes = set(kern_2024['Area_Code'].unique())
    
    print(f"\nUnique buurten in each file:")
    print(f"  buurt_green_metrics: {len(green_codes)} (using regular codes)")
    print(f"  Kerncijfers 2020: {len(kern_2020_codes)} (using CBS codes)")
    print(f"  Kerncijfers 2024: {len(kern_2024_codes)} (using CBS codes)")
    
    # Check if 2020 codes map to 2015 system
    kern_2020_in_2015 = len(kern_2020_codes.intersection(valid_buurt_codes_2015_cbs))
    kern_2020_in_2022 = len(kern_2020_codes.intersection(valid_buurt_codes_2022_cbs))
    
    print(f"\nKerncijfers 2020 codes:")
    print(f"  Using 2015 CBS system: {kern_2020_in_2015} codes")
    print(f"  Using 2022 CBS system: {kern_2020_in_2022} codes")
    
    # Check if 2024 codes map to 2022 system
    kern_2024_in_2015 = len(kern_2024_codes.intersection(valid_buurt_codes_2015_cbs))
    kern_2024_in_2022 = len(kern_2024_codes.intersection(valid_buurt_codes_2022_cbs))
    
    print(f"\nKerncijfers 2024 codes:")
    print(f"  Using 2015 CBS system: {kern_2024_in_2015} codes")
    print(f"  Using 2022 CBS system: {kern_2024_in_2022} codes")
    
    # Check green metrics codes
    green_in_2015 = len(green_codes.intersection(valid_buurt_codes_2015_regular))
    green_in_2022 = len(green_codes.intersection(valid_buurt_codes_2022_regular))
    
    print(f"\nGreen metrics codes:")
    print(f"  Using 2015 regular system: {green_in_2015} codes")
    print(f"  Using 2022 regular system: {green_in_2022} codes")
    
except Exception as e:
    print(f"Could not perform validation: {e}")

print(f"\n✓ All files have been filtered and saved with 'filtered_' prefix")

Loading reference file: buurten.csv
Found 439 valid 2015 CBS buurt codes
Found 439 valid 2022 CBS buurt codes
Total unique valid CBS codes: 878

Found 439 valid 2015 regular buurt codes
Found 439 valid 2022 regular buurt codes
Total unique valid regular codes: 878

Sample CBS codes: ['BU0363AF09', 'BU03636100', 'BU03635806', 'BU0363KD05', 'BU0363AH04']
Sample regular codes: ['ME03', 'A04h', 'FQ10', 'KM01', 'NG03']

Processing: buurt_green_metrics.csv
Original rows: 439
File year: 2022
Using: regular buurt codes
Filtered rows: 439
Removed rows: 0
✓ Saved filtered data to: filtered_buurt_green_metrics.csv

Code distribution:
  Matches 2015 codes: 0
  Matches 2022 codes: 439

Processing: Kerncijfers_wijken_en_buurten_2020.csv
Original rows: 579
File year: 2020
Using: CBS buurt codes
Filtered rows: 439
Removed rows: 140

Sample removed codes (first 10):
  - WK036364 [Wijk (WK)]
  - WK036338 [Wijk (WK)]
  - WK036325 [Wijk (WK)]
  - WK036307 [Wijk (WK)]
  - WK036351 [Wijk (WK)]
  - WK036333 

In [ ]:
print("="*80)
print("STEP 1: Loading all files")
print("="*80)

# Load with correct separators and decimal handling
kern_2020 = pd.read_csv('../data/filtered_Kerncijfers_wijken_en_buurten_2020.csv', 
                         sep=';', decimal=',')
kern_2024 = pd.read_csv('../data/filtered_Kerncijfers_wijken_en_buurten_2024.csv', 
                         sep=';', decimal=',')
green_metrics = pd.read_csv('../data/filtered_buurt_green_metrics.csv')
facilities_2020 = pd.read_csv('../data/filtered_proximity_to_facilities_2020.csv', sep=';', decimal=',')
facilities_2024 = pd.read_csv('../data/filtered_proximity_to_facilities_2024.csv', sep=';', decimal=',')
buurten_mapping = pd.read_csv('../data/buurten.csv')

print(f"Kern 2020: {len(kern_2020)} rows, {kern_2020.shape[1]} columns")
print(f"Kern 2024: {len(kern_2024)} rows, {kern_2024.shape[1]} columns")
print(f"Green: {len(green_metrics)} rows, {green_metrics.shape[1]} columns")
print(f"Facilities 2020: {len(facilities_2020)} rows, {facilities_2020.shape[1]} columns")
print(f"Facilities 2024: {len(facilities_2024)} rows, {facilities_2024.shape[1]} columns")

print("\n" + "="*80)
print("STEP 2: Create mappings")
print("="*80)

# CBS codes to BUURTCODE_2022
cbs2015_to_buurt = dict(zip(buurten_mapping['BUURTCODE_CBS_2015'], 
                            buurten_mapping['BUURTCODE_2022']))
cbs2022_to_buurt = dict(zip(buurten_mapping['BUURTCODE_CBS_2022'], 
                            buurten_mapping['BUURTCODE_2022']))

# Reverse mappings
buurt_to_cbs2015 = dict(zip(buurten_mapping['BUURTCODE_2022'],
                            buurten_mapping['BUURTCODE_CBS_2015']))
buurt_to_cbs2022 = dict(zip(buurten_mapping['BUURTCODE_2022'],
                            buurten_mapping['BUURTCODE_CBS_2022']))

print(f"Mappings created")

print("\n" + "="*80)
print("STEP 3: Define comprehensive column renaming")
print("="*80)

# KERNCIJFERS COLUMNS (same for both 2020 and 2024)
kerncijfers_rename = {
    'District_Type': 'district_type',
    'Municipality_Name': 'municipality_name',
    'Area_Code': 'area_code_original',
    'Population_Total': 'population_total',
    'Pop_Age_0_15': 'pop_age_0_15',
    'Pop_Age_15_25': 'pop_age_15_25',
    'Pop_Age_25_45': 'pop_age_25_45',
    'Pop_Age_45_65': 'pop_age_45_65',
    'Pop_Age_65_Plus': 'pop_age_65_plus',
    'Avg_Household_Size': 'avg_household_size',
    'Avg_Property_Value_1000EUR': 'woz_value',
    'Pct_Unoccupied_Homes': 'pct_unoccupied',
    'Pct_Owner_Occupied': 'pct_owner_occupied',
    'Pct_Rental': 'pct_rental',
    'Higher_Education_Count': 'n_high_education',
    'Avg_Income_Per_Income_Recipient_1000EUR': 'income_recipient',
    'Avg_Income_Per_Inhabitant_1000EUR': 'income_inhabitant',
    'Avg_Std_Household_Income_1000EUR': 'income_household',
    'Pct_Lowest_40Pct_Income': 'pct_bottom40',
    'Pct_Highest_20Pct_Income': 'pct_top20',
    'Pct_Low_Income_Households': 'pct_low_income',
    'Pct_Below_120Pct_Social_Min': 'pct_below_120pct_social_min',
    'Urbanization_Level': 'urbanicity'
}

# FACILITIES 2020 COLUMNS
facilities_2020_rename = {
    'ID': 'facility_id',
    'DistrictsAndNeighbourhoods': 'districts_and_neighbourhoods',
    'MunicipalityName': 'municipality_name_fac',
    'Encoding': 'encoding_original',
    'supermarkets_within_1km': 'supermarkets_1km',
    'food_shops_within_1km': 'food_shops_1km',
    'cafes_within_1km': 'cafes_1km',
    'cafeterias_within_1km': 'cafeterias_1km',
    'restaurants_within_1km': 'restaurants_1km',
    'distance_to_primary_school': 'dist_primary_school',
    'distance_to_secondary_school': 'dist_secondary_school',
    'DistanceToTrainStationAllTypes': 'dist_train_station'
}

# FACILITIES 2024 COLUMNS (note differences in column names)
facilities_2024_rename = {
    'ID': 'facility_id',
    'DistrictsAndNeighbourhoods': 'districts_and_neighbourhoods',
    'MunicipalityName_1': 'municipality_name_fac',
    'Encoding_3': 'encoding_original',
    'supermarkets_within_1km': 'supermarkets_1km',
    'food_shops_within_1km': 'food_shops_1km',
    'cafes_within_1km': 'cafes_1km',
    'cafeterias_within_1km': 'cafeterias_1km',
    'restaurants_within_1km': 'restaurants_1km',
    'distance_to_primary_school': 'dist_primary_school',
    'distance_to_secondary_school': 'dist_secondary_school',
    'DistanceToTrainStationAllTypes_90': 'dist_train_station'
}

print("Column renaming dictionaries defined")

print("\n" + "="*80)
print("STEP 4: Process GREEN METRICS (both years combined)")
print("="*80)

# Extract metadata (same for both years)
green_metadata = green_metrics[['BUURTCODE_2022', 'BUURTNAAM_2022', 'WIJKNAAM_2022', 
                                 'STADSDEELNAAM', 'buurt_area_m2']].copy()
green_metadata = green_metadata.rename(columns={
    'BUURTCODE_2022': 'buurtcode_2022',
    'BUURTNAAM_2022': 'buurt_name',
    'WIJKNAAM_2022': 'wijk_name',
    'STADSDEELNAAM': 'stadsdeel_name',
    'buurt_area_m2': 'buurt_area_m2'
})

# Extract 2020 metrics with year suffix
green_2020 = green_metrics[['BUURTCODE_2022', 
                            'distance_to_nearest_green_m_2020',
                            'nearest_green_name_2020',
                            'nearest_green_type_2020',
                            'nearest_green_area_m2_2020',
                            'total_green_area_m2_2020',
                            'green_coverage_percent_2020',
                            'num_green_spaces_2020']].copy()

green_2020 = green_2020.rename(columns={
    'BUURTCODE_2022': 'buurtcode_2022',
    'distance_to_nearest_green_m_2020': 'dist_nearest_green_2020',
    'nearest_green_name_2020': 'nearest_green_name_2020',
    'nearest_green_type_2020': 'nearest_green_type_2020',
    'nearest_green_area_m2_2020': 'nearest_green_area_m2_2020',
    'total_green_area_m2_2020': 'total_green_area_m2_2020',
    'green_coverage_percent_2020': 'green_coverage_pct_2020',
    'num_green_spaces_2020': 'num_green_spaces_2020'
})

# Extract 2024 metrics with year suffix
green_2024 = green_metrics[['BUURTCODE_2022',
                            'distance_to_nearest_green_m_2024',
                            'nearest_green_name_2024',
                            'nearest_green_type_2024',
                            'nearest_green_area_m2_2024',
                            'total_green_area_m2_2024',
                            'green_coverage_percent_2024',
                            'num_green_spaces_2024']].copy()

green_2024 = green_2024.rename(columns={
    'BUURTCODE_2022': 'buurtcode_2022',
    'distance_to_nearest_green_m_2024': 'dist_nearest_green_2024',
    'nearest_green_name_2024': 'nearest_green_name_2024',
    'nearest_green_type_2024': 'nearest_green_type_2024',
    'nearest_green_area_m2_2024': 'nearest_green_area_m2_2024',
    'total_green_area_m2_2024': 'total_green_area_m2_2024',
    'green_coverage_percent_2024': 'green_coverage_pct_2024',
    'num_green_spaces_2024': 'num_green_spaces_2024'
})

# Merge metadata + 2020 + 2024 into single green dataframe
green_combined = green_metadata.merge(green_2020, on='buurtcode_2022', how='outer')
green_combined = green_combined.merge(green_2024, on='buurtcode_2022', how='outer')

print(f"Green combined: {len(green_combined)} rows, {len(green_combined.columns)} columns")
print(f"Columns: {list(green_combined.columns)}")

print("\n" + "="*80)
print("STEP 5: Process 2020 data (ALL COLUMNS)")
print("="*80)

# Kerncijfers 2020
k20 = kern_2020.copy()
k20 = k20.rename(columns=kerncijfers_rename)
k20['buurtcode_2022'] = k20['area_code_original'].map(cbs2015_to_buurt)
k20['buurt_cbs_2015'] = k20['area_code_original']
k20['buurt_cbs_2022'] = k20['buurtcode_2022'].map(buurt_to_cbs2022)
k20['year'] = 2020

print(f"K20 columns after rename: {len(k20.columns)}")

# Facilities 2020
f20 = facilities_2020.copy()
f20 = f20.rename(columns=facilities_2020_rename)
f20['buurtcode_2022'] = f20['encoding_original'].map(cbs2015_to_buurt)

print(f"F20 columns after rename: {len(f20.columns)}")

# Select green columns for 2020 (metadata + 2020 metrics only)
green_cols_2020 = ['buurtcode_2022', 'buurt_name', 'wijk_name', 'stadsdeel_name', 
                   'buurt_area_m2'] + [col for col in green_combined.columns if '2020' in col]
g20 = green_combined[green_cols_2020].copy()

print(f"G20 columns selected: {len(g20.columns)}")

# Merge 2020 - keep ALL columns
data_2020 = k20.merge(f20.drop(columns=['encoding_original']), 
                       on='buurtcode_2022', how='left', suffixes=('', '_fac'))
data_2020 = data_2020.merge(g20, on='buurtcode_2022', how='left')

print(f"2020 data: {len(data_2020)} rows, {len(data_2020.columns)} columns")

print("\n" + "="*80)
print("STEP 6: Process 2024 data (ALL COLUMNS)")
print("="*80)

# Kerncijfers 2024
k24 = kern_2024.copy()
k24 = k24.rename(columns=kerncijfers_rename)
k24['buurtcode_2022'] = k24['area_code_original'].map(cbs2022_to_buurt)
k24['buurt_cbs_2022'] = k24['area_code_original']
k24['buurt_cbs_2015'] = k24['buurtcode_2022'].map(buurt_to_cbs2015)
k24['year'] = 2024

print(f"K24 columns after rename: {len(k24.columns)}")

# Facilities 2024
f24 = facilities_2024.copy()
f24 = f24.rename(columns=facilities_2024_rename)
f24['buurtcode_2022'] = f24['encoding_original'].map(cbs2022_to_buurt)

print(f"F24 columns after rename: {len(f24.columns)}")

# Select green columns for 2024 (metadata + 2024 metrics only)
green_cols_2024 = ['buurtcode_2022', 'buurt_name', 'wijk_name', 'stadsdeel_name', 
                   'buurt_area_m2'] + [col for col in green_combined.columns if '2024' in col]
g24 = green_combined[green_cols_2024].copy()

print(f"G24 columns selected: {len(g24.columns)}")

# Merge 2024 - keep ALL columns
data_2024 = k24.merge(f24.drop(columns=['encoding_original']), 
                       on='buurtcode_2022', how='left', suffixes=('', '_fac'))
data_2024 = data_2024.merge(g24, on='buurtcode_2022', how='left')

print(f"2024 data: {len(data_2024)} rows, {len(data_2024.columns)} columns")

print("\n" + "="*80)
print("STEP 7: Combine to long format")
print("="*80)

# Stack
final_data = pd.concat([data_2020, data_2024], ignore_index=True)

# Remove missing buurtcodes
final_data = final_data.dropna(subset=['buurtcode_2022'])

print(f"Final: {len(final_data)} rows")
print(f"Unique buurten: {final_data['buurtcode_2022'].nunique()}")
print(f"Total columns: {len(final_data.columns)}")
print(f"Rows by year:\n{final_data['year'].value_counts().sort_index()}")

print("\n" + "="*80)
print("STEP 8: Column inventory")
print("="*80)

print("\nAll columns in final dataset:")
for i, col in enumerate(final_data.columns, 1):
    print(f"  {i:2d}. {col}")

# Identify green columns
green_2020_cols_in_final = [col for col in final_data.columns if '2020' in col]
green_2024_cols_in_final = [col for col in final_data.columns if '2024' in col]

print(f"\nGreen metrics 2020 columns ({len(green_2020_cols_in_final)}):")
for col in green_2020_cols_in_final:
    print(f"  - {col}")

print(f"\nGreen metrics 2024 columns ({len(green_2024_cols_in_final)}):")
for col in green_2024_cols_in_final:
    print(f"  - {col}")

print("\n" + "="*80)
print("STEP 9: Data quality check")
print("="*80)

print("\nMissing values per column:")
missing = final_data.isnull().sum()
missing_pct = (missing / len(final_data) * 100).round(1)
missing_df = pd.DataFrame({
    'Missing': missing[missing > 0],
    'Percent': missing_pct[missing > 0]
}).sort_values('Missing', ascending=False)

if len(missing_df) > 0:
    print(missing_df.to_string())
else:
    print("No missing values!")

print("\nSample (first 2 rows each year):")
sample_cols = ['buurtcode_2022', 'year', 'buurt_name', 'wijk_name', 
               'income_inhabitant', 'green_coverage_pct_2020', 'green_coverage_pct_2024',
               'woz_value', 'restaurants_1km', 'dist_train_station']
available_cols = [col for col in sample_cols if col in final_data.columns]
print(final_data.groupby('year').head(2)[available_cols].to_string(index=False))

print("\n" + "="*80)
print("STEP 10: Save")
print("="*80)

final_data.to_csv('../data/merged_buurt_panel_data.csv', index=False)
print(f"✓ Saved: merged_buurt_panel_data.csv")
print(f"  Shape: {final_data.shape}")
print(f"  Columns: {len(final_data.columns)}")
print(f"  Rows per year: 2020={len(final_data[final_data['year']==2020])}, 2024={len(final_data[final_data['year']==2024])}")

print("\n" + "="*80)
print("VERIFICATION: Check both years' green metrics are present")
print("="*80)
print(f"2020 rows have 2020 green data: {final_data[final_data['year']==2020]['green_coverage_pct_2020'].notna().sum()}/{len(final_data[final_data['year']==2020])}")
print(f"2020 rows have 2024 green data: {final_data[final_data['year']==2020]['green_coverage_pct_2024'].notna().sum()}/{len(final_data[final_data['year']==2020])}")
print(f"2024 rows have 2020 green data: {final_data[final_data['year']==2024]['green_coverage_pct_2020'].notna().sum()}/{len(final_data[final_data['year']==2024])}")
print(f"2024 rows have 2024 green data: {final_data[final_data['year']==2024]['green_coverage_pct_2024'].notna().sum()}/{len(final_data[final_data['year']==2024])}")

STEP 1: Loading all files
Kern 2020: 439 rows, 23 columns
Kern 2024: 439 rows, 23 columns
Green: 439 rows, 21 columns
Facilities 2020: 439 rows, 12 columns
Facilities 2024: 439 rows, 12 columns

STEP 2: Create mappings
Mappings created

STEP 3: Define comprehensive column renaming
Column renaming dictionaries defined

STEP 4: Process GREEN METRICS (both years combined)
Green combined: 439 rows, 19 columns
Columns: ['buurtcode_2022', 'buurt_name', 'wijk_name', 'stadsdeel_name', 'buurt_area_m2', 'dist_nearest_green_2020', 'nearest_green_name_2020', 'nearest_green_type_2020', 'nearest_green_area_m2_2020', 'total_green_area_m2_2020', 'green_coverage_pct_2020', 'num_green_spaces_2020', 'dist_nearest_green_2024', 'nearest_green_name_2024', 'nearest_green_type_2024', 'nearest_green_area_m2_2024', 'total_green_area_m2_2024', 'green_coverage_pct_2024', 'num_green_spaces_2024']

STEP 5: Process 2020 data (ALL COLUMNS)
K20 columns after rename: 27
F20 columns after rename: 13
G20 columns selected

Final: 878 rows
Unique buurten: 439
Total columns: 56
Rows by year:
year
2020    439
2024    439
Name: count, dtype: int64

STEP 8: Column inventory

All columns in final dataset:
   1. district_type
   2. municipality_name
   3. area_code_original
   4. population_total
   5. pop_age_0_15
   6. pop_age_15_25
   7. pop_age_25_45
   8. pop_age_45_65
   9. pop_age_65_plus
  10. avg_household_size
  11. woz_value
  12. pct_unoccupied
  13. pct_owner_occupied
  14. pct_rental
  15. n_high_education
  16. income_recipient
  17. income_inhabitant
  18. income_household
  19. pct_bottom40
  20. pct_top20
  21. pct_low_income
  22. pct_below_120pct_social_min
  23. urbanicity
  24. buurtcode_2022
  25. buurt_cbs_2015
  26. buurt_cbs_2022
  27. year
  28. facility_id
  29. districts_and_neighbourhoods
  30. municipality_name_fac
  31. supermarkets_1km
  32. food_shops_1km
  33. cafes_1km
  34. cafeterias_1km
  35. restaurants_1km
  36. dist_primary_school
  37. dist_secondary_school
  38. dist_

### remove leading whitespaces

In [ ]:
# Load data
df = pd.read_csv('../data/merged_buurt_panel_data.csv')

# Strip whitespace from all string columns
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Save cleaned data
df.to_csv('../data/merged_buurt_panel_data.csv', index=False)

print("✓ Removed leading/trailing whitespaces")
print(f"Shape: {df.shape}")

✓ Removed leading/trailing whitespaces
Shape: (878, 56)


### now save a copy of the merged_buurt_panel_data.csv file to be able to run tests on the non-imputed dataset later

In [ ]:
# Load the current merged_buurt_panel_data.csv
df = pd.read_csv('../data/merged_buurt_panel_data.csv')

# Save a copy with the new name
df.to_csv('../data/non_imputed_merged_buurt_panel_data.csv', index=False)

print("✓ Saved copy as non_imputed_merged_buurt_panel_data.csv")

✓ Saved copy as non_imputed_merged_buurt_panel_data.csv


## 🧮 account for missing values: KNN imputation strategy

We handle missing values using **context-aware KNN imputation**, applied in **three separate batches** to reduce bias and avoid circular leakage between related socioeconomic variables.

### 🔹 Overview
Instead of imputing all variables at once, we impute them in **groups** with contextually relevant features:
- Each batch has its own **target variables** (to be imputed)
- and **context features** (used to find similar neighborhoods)

This approach ensures that:
- Income variables aren’t imputed using other income measures directly  
- Housing variables are informed by spatial and demographic context  
- Education and household size are imputed using population and income characteristics  

### 🔹 Batches
1. **Housing characteristics** → `woz_value`, `pct_unoccupied`, `pct_owner_occupied`, `pct_rental`  
   *Context:* urbanicity, population, facilities, green area, etc.  

2. **Income-related** → `income_household`, `income_recipient`, `income_inhabitant`, `pct_top20`, `pct_bottom40`, `pct_low_income`, `pct_below_120pct_social_min`  
   *Context:* urbanicity, education, housing, and population structure  

3. **Education & household composition** → `n_high_education`, `avg_household_size`  
   *Context:* income, age structure, urbanicity  

### 🔹 Implementation Notes
- Uses `KNNImputer(n_neighbors=5, weights='distance')`  
- All variables converted to numeric (`errors='coerce'`)  
- Each batch validated with summary statistics (mean, median, min, max)  
- Original data saved as `non_imputed_merged_buurt_panel_data.csv` for comparison  
- Final imputed dataset exported as `merged_buurt_panel_data_imputed.csv`  

This modular setup preserves interpretability and statistical validity for subsequent analysis (e.g., DiD regression).




Why KNN imputation instead of median?

KNN uses similar buurten to fill values (e.g., if Buurt A is missing pct_low_income, it looks at the 5 most similar buurten based on urbanicity, population, green coverage, etc.)
More accurate than simple median
Preserves relationships between variables

In [ ]:
from sklearn.impute import KNNImputer

def knn_impute(df, target_vars, context_vars, k=5):
    """
    KNN imputation with robust handling of column order, duplicate names and shape checks.
    Replaces only the target_vars in the original df.
    """
    # available columns
    available_targets = [c for c in target_vars if c in df.columns]
    available_context = [c for c in context_vars if c in df.columns]

    if not available_targets:
        print("  ⚠️  WARNING: No target variables found in dataframe!")
        return df

    missing_targets = set(target_vars) - set(available_targets)
    missing_context = set(context_vars) - set(available_context)
    if missing_targets:
        print(f"  ⚠️  WARNING: Target variables not found: {missing_targets}")
    if missing_context:
        print(f"  ⚠️  WARNING: Context variables not found: {missing_context}")

    # preserve order and remove duplicates
    all_vars = []
    for c in available_targets + available_context:
        if c not in all_vars:
            all_vars.append(c)

    # select columns (preserve dataframe column order if possible)
    impute_df = df.loc[:, [c for c in all_vars if c in df.columns]].copy()

    # drop duplicated column names if any (rare, but possible)
    if impute_df.columns.duplicated().any():
        dup = impute_df.columns[impute_df.columns.duplicated()].tolist()
        print(f"  ⚠️  Dropping duplicate column names: {dup}")
        impute_df = impute_df.loc[:, ~impute_df.columns.duplicated()]

    # convert to numeric (coerce strings to NaN)
    for col in impute_df.columns:
        impute_df[col] = pd.to_numeric(impute_df[col], errors='coerce')

    # debug info
    print(f"  Imputation matrix shape (rows,cols): {impute_df.shape}")
    print(f"  Columns used: {list(impute_df.columns)}")
    
    # Show missing values before imputation
    print(f"  Missing values in batch before imputation:")
    missing_counts = impute_df.isnull().sum()
    for target in available_targets:
        if target in missing_counts.index:
            print(f"    {target}: {missing_counts[target]}")

    # run imputer on numpy array
    imputer = KNNImputer(n_neighbors=k, weights='distance')
    arr = impute_df.to_numpy(dtype=float)
    imputed_arr = imputer.fit_transform(arr)

    print(f"  Imputed array shape: {imputed_arr.shape}")

    # ensure column alignment; handle rare mismatch robustly
    if imputed_arr.shape[1] != impute_df.shape[1]:
        print("  ⚠️  Shape mismatch between imputed array and impute_df.columns.")
        cols = list(impute_df.columns)
        if imputed_arr.shape[1] < len(cols):
            cols = cols[:imputed_arr.shape[1]]
            print(f"  ⚠️  Trimming column list to: {cols}")
        else:
            extras = [f"imputed_extra_{i}" for i in range(imputed_arr.shape[1] - len(cols))]
            cols = cols + extras
            print(f"  ⚠️  Extending column list with: {extras}")
        imputed_df = pd.DataFrame(imputed_arr, columns=cols, index=df.index)
    else:
        imputed_df = pd.DataFrame(imputed_arr, columns=impute_df.columns, index=df.index)

    # replace only the target variables (if present in imputed_df)
    for var in available_targets:
        if var in imputed_df.columns:
            df[var] = imputed_df[var]
        else:
            print(f"  ⚠️  Imputed result does not contain column '{var}' — skipping update.")

    return df


# ============================================================
# MAIN EXECUTION
# ============================================================

print("="*80)
print("CONTEXT-AWARE BATCH KNN IMPUTATION")
print("="*80)

df = pd.read_csv('../data/merged_buurt_panel_data.csv', na_values=['.', '..', ''])

print(f"\nTotal rows: {len(df)}")
print(f"Total columns: {len(df.columns)}")
print(f"\nRows by year:\n{df['year'].value_counts().sort_index()}")

print("\n" + "="*80)
print("Missing values before imputation (all relevant variables):")
print("="*80)
relevant_vars = [
    'woz_value', 'pct_unoccupied', 'pct_owner_occupied', 'pct_rental',
    'income_household', 'income_recipient', 'income_inhabitant',
    'pct_top20', 'pct_bottom40', 'pct_below_120pct_social_min', 'pct_low_income',
    'n_high_education', 'avg_household_size'
]
missing_before = df[relevant_vars].isnull().sum()
print(missing_before[missing_before > 0].sort_values(ascending=False))

# ============================================================
# BATCH 1: Housing characteristics
# ============================================================
print("\n" + "="*80)
print("BATCH 1: Housing Characteristics")
print("="*80)

batch1_targets = ['woz_value', 'pct_unoccupied', 'pct_owner_occupied', 'pct_rental']
batch1_context = [
    'urbanicity',
    'population_total', 'avg_household_size',
    'dist_train_station', 'dist_primary_school',
    'supermarkets_1km', 'restaurants_1km',
    'year'
]

print(f"\nTargets: {batch1_targets}")
print(f"Context features: {len(batch1_context)} features")

df = knn_impute(df, batch1_targets, batch1_context, k=5)

print(f"\n  Validation - Imputed housing variables:")
for var in batch1_targets:
    if var in df.columns:
        print(f"\n  {var}:")
        print(f"    Mean: {df[var].mean():.2f}")
        print(f"    Median: {df[var].median():.2f}")
        print(f"    Min: {df[var].min():.2f}")
        print(f"    Max: {df[var].max():.2f}")
        print(f"    Still missing: {df[var].isnull().sum()}")

# ============================================================
# BATCH 2: Income-related variables
# ============================================================
print("\n" + "="*80)
print("BATCH 2: Income-Related Variables")
print("="*80)

batch2_targets = [
    'income_household', 'income_recipient', 'income_inhabitant',
    'pct_top20', 'pct_bottom40', 'pct_below_120pct_social_min', 'pct_low_income'
]
batch2_context = [
    'urbanicity',
    'woz_value', 'n_high_education', 'avg_household_size',
    'pct_owner_occupied', 'pct_rental', 'population_total', 'year'
]

print(f"\nTargets: {batch2_targets}")
print(f"Context features: {len(batch2_context)} features")

df = knn_impute(df, batch2_targets, batch2_context, k=5)

print(f"\n  Validation - Imputed income variables:")
for var in batch2_targets:
    if var in df.columns:
        print(f"\n  {var}:")
        print(f"    Mean: {df[var].mean():.2f}")
        print(f"    Median: {df[var].median():.2f}")
        print(f"    Min: {df[var].min():.2f}")
        print(f"    Max: {df[var].max():.2f}")
        print(f"    Still missing: {df[var].isnull().sum()}")

# ============================================================
# BATCH 3: Education and household composition
# ============================================================
print("\n" + "="*80)
print("BATCH 3: Education and Household Composition")
print("="*80)

batch3_targets = ['n_high_education', 'avg_household_size']
batch3_context = [
    'urbanicity',
    'income_household', 'woz_value',
    'population_total', 'pop_age_25_45', 'pop_age_45_65', 'pop_age_65_plus', 'year'
]

print(f"\nTargets: {batch3_targets}")
print(f"Context features: {len(batch3_context)} features")

df = knn_impute(df, batch3_targets, batch3_context, k=5)

print(f"\n  Validation - Imputed education/household variables:")
for var in batch3_targets:
    if var in df.columns:
        print(f"\n  {var}:")
        print(f"    Mean: {df[var].mean():.2f}")
        print(f"    Median: {df[var].median():.2f}")
        print(f"    Min: {df[var].min():.2f}")
        print(f"    Max: {df[var].max():.2f}")
        print(f"    Still missing: {df[var].isnull().sum()}")

# ============================================================
# BATCH 4: Facility variables
# ============================================================
print("\n" + "="*80)
print("BATCH 4: Facility Variables")
print("="*80)

facility_vars = [
    'restaurants_1km',
    'cafes_1km',
    'cafeterias_1km',
    'dist_primary_school',
    'dist_train_station',
    'supermarkets_1km',
    'food_shops_1km',
    'dist_secondary_school',
]
facility_context = [
    'urbanicity',
    'population_total',
    'avg_household_size',
    'woz_value',
    'income_household',
    'year'
]

print(f"\nTargets: {facility_vars}")
print(f"Context features: {len(facility_context)} features")

df = knn_impute(df, facility_vars, facility_context, k=5)

print(f"\n  Validation - Imputed facility variables:")
for var in facility_vars:
    if var in df.columns:
        print(f"\n  {var}:")
        print(f"    Mean: {df[var].mean():.2f}")
        print(f"    Median: {df[var].median():.2f}")
        print(f"    Min: {df[var].min():.2f}")
        print(f"    Max: {df[var].max():.2f}")
        print(f"    Still missing: {df[var].isnull().sum()}")

# ============================================================
# FINAL SUMMARY AND VALIDATION
# ============================================================
print("\n" + "="*80)
print("FINAL SUMMARY: Missing values after all batches")
print("="*80)

missing_after = df.isnull().sum()
print(missing_after[missing_after > 0].sort_values(ascending=False))

print("\n" + "="*80)
print("Comparing with non-imputed dataset")
print("="*80)

try:
    df_original = pd.read_csv('non_imputed_merged_buurt_panel_data.csv', na_values=['.', '..', ''])
    
    # Combine all imputed variables for comparison
    all_imputed_vars = relevant_vars + facility_vars
    
    print("\nImputation effectiveness:")
    for var in all_imputed_vars:
        if var in df.columns and var in df_original.columns:
            before_missing = df_original[var].isnull().sum()
            after_missing = df[var].isnull().sum()
            improvement = before_missing - after_missing
            if before_missing > 0:
                print(f"  {var}:")
                print(f"    Before: {before_missing} missing ({100*before_missing/len(df):.1f}%)")
                print(f"    After: {after_missing} missing ({100*after_missing/len(df):.1f}%)")
                print(f"    Improvement: {improvement} values imputed ({100*improvement/before_missing:.1f}% of missing)")
except FileNotFoundError:
    print("  Non-imputed file not found for comparison")

# ============================================================
# SAVE IMPUTED DATASET
# ============================================================
print("\n" + "="*80)
print("Saving imputed dataset")
print("="*80)

df.to_csv('../data/merged_buurt_panel_data_imputed.csv', index=False)
print("✓ Saved: merged_buurt_panel_data_imputed.csv")

print(f"\nFinal dataset:")
print(f"  Rows: {len(df)}")
print(f"  Columns: {len(df.columns)}")
print(f"  Complete cases (no missing): {df.dropna().shape[0]}")
print(f"  Rows by year:\n{df['year'].value_counts().sort_index()}")

print("\n" + "="*80)
print("Ready for DiD analysis!")
print("="*80)

all_targets = batch1_targets + batch2_targets + batch3_targets + facility_vars
complete_vars = [v for v in all_targets if v in df.columns and df[v].notna().all()]
incomplete_vars = [v for v in all_targets if v in df.columns and df[v].isnull().any()]

print(f"\nVariables status:")
print(f"  Complete (no missing): {complete_vars}")
print(f"  Incomplete (still missing): {incomplete_vars}")
print(f"\nKey variables for analysis:")
print(f"  Treatment: green_coverage_pct_2020, green_coverage_pct_2024")
print(f"  Outcomes: income_household, income_inhabitant, pct_low_income, woz_value")
print(f"  Controls: urbanicity, population_total, facilities, housing characteristics")

CONTEXT-AWARE BATCH KNN IMPUTATION

Total rows: 878
Total columns: 56

Rows by year:
year
2020    439
2024    439
Name: count, dtype: int64

Missing values before imputation (all relevant variables):
n_high_education               207
woz_value                      121
pct_unoccupied                 108
pct_rental                     108
pct_owner_occupied             108
income_household                70
income_recipient                52
income_inhabitant               46
avg_household_size              16
pct_bottom40                    12
pct_top20                       12
pct_low_income                  12
pct_below_120pct_social_min     12
dtype: int64

BATCH 1: Housing Characteristics

Targets: ['woz_value', 'pct_unoccupied', 'pct_owner_occupied', 'pct_rental']
Context features: 8 features
  Imputation matrix shape (rows,cols): (878, 12)
  Columns used: ['woz_value', 'pct_unoccupied', 'pct_owner_occupied', 'pct_rental', 'urbanicity', 'population_total', 'avg_household_size', 'd